In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
! pip install google-genai

  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


In [9]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
import textwrap
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForTokenClassification, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoModelForSequenceClassification, Trainer, EarlyStoppingCallback
import os
from datasets import load_dataset, Dataset, DatasetDict
import transformers
from trl import SFTTrainer
import bitsandbytes as bnb
import evaluate
import random
import pandas as pd
import base64
from IPython.display import Image, display, Markdown
import ipywidgets as widgets
from sklearn.metrics import classification_report
# from google.colab import userdata
import openai
from openai import Client
from copy import deepcopy
import json
import backoff
import csv
import time
import nltk
from nltk.tokenize import sent_tokenize
from lexrank import LexRank
from humanfriendly import format_timespan
from tqdm.notebook import tqdm
from alive_progress import alive_bar
from functools import partialmethod
from IPython.display import clear_output
import sys
import sentencepiece
import re
import string
from google import genai
from google.genai import types

In [10]:
with open('api/config.json') as f:
    config = json.load(f)

In [11]:
REPLICATE_API_TOKEN = config['REPLICATE_API_TOKEN']
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
OPENAI_API_KEY = config['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
KIMI_API_KEY = config['KIMI_API_KEY']
os.environ["KIMI_API_KEY"] = KIMI_API_KEY
GEMINI_API_KEY = config['GEMINI_API_KEY']
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [12]:
# import torch
torch.cuda.is_available()

False

In [6]:
def normalize(scores):
    return (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

In [7]:
from global_token_attribution2.sota_models.DecompX.src.decompx_utils import DecompXConfig
from global_token_attribution2.sota_models.DecompX.src.modeling_bert import BertForSequenceClassification
from global_token_attribution2.sota_models.DecompX.src.modeling_roberta import RobertaForSequenceClassification

In [8]:
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)

True

In [9]:
MODEL = "piturrolfio/bert-classifier-maths" # ctu-aic/xlm-roberta-large-nli-qacg-sum # achimoraites/roberta-base_ag_news "Bhumika/roberta-base-finetuned-sst2"  # Only BERT or RoBERTa

SENTENCE = "This is an example document for calculating token attribution."
# SENTENCE = "Focus on the positive in people and situations. I've talked about being positive at work, but it's important in all parts of life. People want to be around those who make them feel good. Think about the people you enjoy spending time with. They probably make you feel happy, give you compliments, or lend a helping hand. They likely don't complain much or put you down. Being happy is a big part of being someone people want to be around. I'm not talking about fake happiness, but genuine joy. It took me years to become a truly happy person, and I still have to work at it. But since I've made this change, it's been easier to make friends and have a busy social life.  I used to struggle to find people to hang out with, but now my social calendar is as full as I want it to be. It's not easy for me, since I'm naturally cynical, but I've made a lot of progress. I'm still working on smiling more, seeing the good in people, listening more than talking, and finding the positive in every situation. It's a journey, but it's worth it. I used to think I was smart, but now I realize there are many people far more intelligent than me. I've been lucky to work with amazing people, and I'm constantly impressed by their brilliance. Sometimes I feel like I've tricked everyone into thinking I'm smarter than I am. But it wasn't luck or intelligence that got me where I am today. It was hard work, long hours, and dedication. I believe you can learn anything you set your mind to. I grew up with a single mother who worked as a waitress. I had to take care of my younger sister and started working as soon as I could to help with expenses. I realized early on that I had to create my own opportunities. At 14, I made a plan for my life – an ambitious plan that I'm still working towards. Every success I've had is a result of hard work, some luck, and the support of others. If you want something, go for it! But remember to keep secrets and never say anything bad about others. It will likely get back to them. I used to gossip because I wanted to have something to say and win people over. Now I realize how foolish that was. Relationships are more important than being right or getting your way. You might forget specific moments or projects, but you'll always remember the people in your life and how they made you feel. Focus on making those interactions positive. Listen more than you talk, be interested in others, and ask questions. A friend once told me to remember a few things that are important to each person I know and ask them about those interests. People appreciate it when you remember these details. Treat everyone with respect. Growing up with a mother who was a waitress taught me that everyone deserves respect.  Even if someone is rude, try to be understanding and kind. I'm not always good at this myself, but I'm working on it. I try to stay positive and be the best version of myself. This mindset of respect extends beyond just how you treat people in the service industry. It's about recognizing the inherent worth of every individual, regardless of their profession, background, or how they treat you.  Everyone has their own struggles and stories, and a little empathy goes a long way.  It's about choosing to rise above negativity and pettiness, and instead, lead with kindness and understanding. This doesn't mean being a pushover, but rather, approaching every interaction with a generous spirit. Ultimately, it's about striving to be the kind of person you want to be, someone who radiates positivity and uplifts those around them.  It's a continuous journey of self-improvement, and there will be times when you fall short. But by consistently reminding yourself of these principles and making a conscious effort to embody them, you'll find yourself attracting positive relationships and creating a more fulfilling life. And who knows, you might even inspire others to do the same."

CONFIGS = {
    "DecompX": DecompXConfig(
        include_biases=True,
        bias_decomp_type="absdot",
        include_LN1=True,
        include_FFN=True,
        FFN_approx_type="GeLU_ZO",
        include_LN2=True,
        aggregation="vector",
        include_classifier_w_pooler=True,
        tanh_approx_type="ZO",
        output_all_layers=True,
        output_attention=None,
        output_res1=None,
        output_LN1=None,
        output_FFN=None,
        output_res2=None,
        output_encoder=None,
        output_aggregated="norm",
        output_pooler="norm",
        output_classifier=True,
    ),
}

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
def calculate_attribution_scores(model, tokenized_text, config):
    """Calculates attribution scores for a tokenized text."""
    with torch.no_grad():
        model.eval()
        _, _, decompx_last_layer_outputs, _ = model(
            **tokenized_text,
            output_attentions=False,
            return_dict=False,
            output_hidden_states=True,
            decompx_config=config
        )
    attribution_scores = decompx_last_layer_outputs.classifier[0, :, 0].cpu().numpy()
    attribution_scores = (attribution_scores - np.min(attribution_scores)) / (
        np.max(attribution_scores) - np.min(attribution_scores)
    )
    return attribution_scores

def tokenize_sentences(text):
    """Splits the text into sentences."""
    return nltk.sent_tokenize(text)

def get_attribution_scores(sentences, tokenizer, model, config):
    """Calculates attribution scores for a list of sentences."""
    all_attribution_scores = []
    all_tokens = []
    for sentence in sentences:
        tokenized_sentence = tokenizer(
            [sentence], return_tensors="pt", padding=True, truncation=True
        ).to(device)
        attribution_scores = calculate_attribution_scores(
            model, tokenized_sentence, config
        )
        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"][0])
        all_attribution_scores.extend(attribution_scores)
        all_tokens.extend(tokens)
    return all_tokens, all_attribution_scores

def get_attribution_scores_with_sentence_importance(sentences, tokenizer, model, config):
    """Calculates attribution scores with sentence weighting."""

    # Calculate sentence importance scores for sentences using LexRank
    lexrank = LexRank(" ".join(sentences))
    sentence_scores = lexrank.rank_sentences(sentences)

    all_attribution_scores = []
    all_tokens = []
    for i, sentence in enumerate(sentences):
        tokenized_sentence = tokenizer(
            [sentence], return_tensors="pt", padding=True, truncation=True
        ).to(device)
        attribution_scores = calculate_attribution_scores(
            model, tokenized_sentence, config
        )
        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"][0])

        # Apply sentence weight
        attribution_scores *= sentence_scores[i]

        all_attribution_scores.extend(attribution_scores)
        all_tokens.extend(tokens)

    return all_tokens, all_attribution_scores

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# tokenized_sentence = tokenizer([SENTENCE], return_tensors="pt", padding=True)

if "roberta" in MODEL:
    model = RobertaForSequenceClassification.from_pretrained(MODEL, device_map="cuda:0")
elif "bert" in MODEL:
    model = BertForSequenceClassification.from_pretrained(MODEL, device_map="cuda:0")
else:
    raise Exception(f"Not implemented model: {MODEL}")
asd = model.to(device)

In [66]:
sentences = tokenize_sentences(SENTENCE)

In [77]:
# with torch.no_grad():
#     model.eval()
#     _, _, decompx_last_layer_outputs, _ = model(
#         **tokenized_sentence,
#         output_attentions=False,
#         return_dict=False,
#         output_hidden_states=True,
#         decompx_config=CONFIGS["DecompX"]
#     )

In [78]:
# tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"][0])

In [79]:
# attribution_scores = decompx_last_layer_outputs.classifier[0, :, 0].cpu().numpy() # Considering label 0

In [103]:
tokens, attribution_scores = get_attribution_scores(sentences, tokenizer, model, CONFIGS["DecompX"])

In [104]:
print(tokens)

['<s>', '▁This', '▁is', '▁an', '▁example', '▁document', '▁for', '▁calcula', 'ting', '▁to', 'ken', '▁at', 'tribution', '.', '</s>']


In [81]:
# attribution_scores = (attribution_scores - np.min(attribution_scores)) / (
#     np.max(attribution_scores) - np.min(attribution_scores)
# )

In [105]:
tokens_copy = deepcopy(tokens[:])
attribution_scores_copy = deepcopy(attribution_scores[:])
# for i in range(len(tokens_copy)):
i = 0
while i < len(tokens_copy):
    # print("asdsd")
    if i < len(tokens) and tokens[i][0] != "▁" and tokens[i] not in ["[CLS]", "[SEP]", "<s>", "</s>"] and tokens[i] not in string.punctuation:
        # print(tokens)
        # print(i)
        # print(tokens[i-1], tokens[i])
        tokens[i - 1] = tokens[i - 1] + tokens[i][0:]
        # print(tokens[i-1], tokens[i])
        attribution_scores[i - 1] = attribution_scores[i - 1] + attribution_scores[i]
        tokens = np.delete(tokens, i)
        attribution_scores = np.delete(attribution_scores, i)
        i -= 1
        # print(tokens)
        # print(i)
    i += 1
print("Token = Attribution Score")
print("=========================")
for token, score in zip(tokens, attribution_scores):
    if token in ["[CLS]", "[SEP]", "<s>", "</s>"]:
        continue
    print(f"{token} = {score:.4f}")

▁calcula ting
▁calculating ting
▁to ken
▁token ken
▁at tribution
▁attribution tribution
Token = Attribution Score
▁This = 1.0000
▁is = 0.7257
▁an = 0.7244
▁example = 0.9002
▁document = 0.5715
▁for = 0.5651
▁calculating = 1.6551
▁token = 0.3985
▁attribution = 0.9057
. = 0.5102


In [82]:
print("Token = Attribution Score")
print("=========================")
for token, score in zip(tokens, attribution_scores):
    if token in ["<s>", "</s>"]:
        continue
    print(f"{token[1:] if token[0] == 'Ġ' else token} = {score:.4f}")

Token = Attribution Score
[CLS] = 0.0000
This = 0.2744
is = 0.2974
an = 0.3236
exam = 0.3848
##ple = 0.3342
document = 0.6331
for = 0.3128
calcula = 0.5714
##ting = 0.4016
to = 0.2564
##k = 0.2477
##en = 0.3247
at = 0.2324
##trib = 1.0000
##uti = 0.3905
##on = 0.2276
. = 0.2941
[SEP] = 0.0989


In [15]:
score_token_pairs = sorted(list(zip(attribution_scores, tokens, [i for i in range(len(tokens))])), reverse=True)
idx = int(round((len(tokens) * 80.0) / 100.0, 0))
filtered_score_token_pairs = score_token_pairs[:idx]
filtered_tokens = [t[1:] if t[0] == 'Ġ' else t for s, t, i in sorted(filtered_score_token_pairs, key = lambda x: x[2])]
# filtered_tokens.remove("<s>")
filtered_tokens = list(filter(("<s>").__ne__, filtered_tokens))
# filtered_tokens.remove("</s>")
filtered_tokens = list(filter(("</s>").__ne__, filtered_tokens))
print(' '.join(filtered_tokens))

This is an for calculating token attribution .


In [30]:
score_token_pairs = sorted(list(zip(attribution_scores, tokens, [i for i in range(len(tokens))])), reverse=True)
idx = int(round((len(tokens) * 80.0) / 100.0, 0))
filtered_score_token_pairs = score_token_pairs[:idx]
filtered_tokens = [t[1:] if t[0] == 'Ġ' else t for s, t, i in sorted(filtered_score_token_pairs, key = lambda x: x[2])]
# filtered_tokens.remove("<s>")
filtered_tokens = list(filter(("<s>").__ne__, filtered_tokens))
# filtered_tokens.remove("</s>")
filtered_tokens = list(filter(("</s>").__ne__, filtered_tokens))
print(' '.join(filtered_tokens))

This is an document calculating token attribution .


In [202]:
score_token_pairs = sorted(list(zip(attribution_scores, tokens, [i for i in range(len(tokens))])), reverse=True)
idx = int(round((len(tokens) * 80.0) / 100.0, 0))
filtered_score_token_pairs = score_token_pairs[:idx]
filtered_tokens = [t[1:] if t[0] == 'Ġ' else t for s, t, i in sorted(filtered_score_token_pairs, key = lambda x: x[2])]
# filtered_tokens.remove("<s>")
filtered_tokens = list(filter(("<s>").__ne__, filtered_tokens))
# filtered_tokens.remove("</s>")
filtered_tokens = list(filter(("</s>").__ne__, filtered_tokens))
print(' '.join(filtered_tokens))

This is an example document for calculating .


In [13]:
def get_scores_decompx(_text):
    _tokenizer = AutoTokenizer.from_pretrained(MODEL)
    _sentences = tokenize_sentences(_text)
    _tokens, _attribution_scores = get_attribution_scores(_sentences, _tokenizer, model, CONFIGS["DecompX"])
    tokens_copy = deepcopy(_tokens[:])
    attribution_scores_copy = deepcopy(_attribution_scores[:])
    # for i in range(len(tokens_copy)):
    i = 0
    while i < len(tokens_copy):
        # if i < len(_tokens) and _tokens[i][:2] == "##":
        if i < len(_tokens) and _tokens[i][0] != "▁" and _tokens[i] not in ["[CLS]", "[SEP]", "<s>", "</s>"] and _tokens[i] not in string.punctuation:
            # print(tokens)
            # print(i)
            # _tokens[i - 1] = _tokens[i - 1] + _tokens[i][2:]
            _tokens[i - 1] = _tokens[i - 1] + _tokens[i][0:]
            _attribution_scores[i - 1] = _attribution_scores[i - 1] + _attribution_scores[i]
            _tokens = np.delete(_tokens, i)
            _attribution_scores = np.delete(_attribution_scores, i)
            i -= 1
            # print(tokens)
            # print(i)
        i += 1
    # print("Token = Attribution Score")
    # print("=========================")
    for token, score in zip(_tokens, _attribution_scores):
        if token in ["[CLS]", "[SEP]", "<s>", "</s>"]:
            continue
        # print(f"{token} = {score:.4f}")
    _score_token_pairs = sorted(list(zip(_attribution_scores, _tokens, [i for i in range(len(_tokens))])), reverse=True)
    # _idx = int(round((len(_tokens) * percentage) / 100.0, 0))
    # _filtered_score_token_pairs = _score_token_pairs[:_idx]
    _clean_score_token_pairs = [(s, t[1:] if t[0] in ['Ġ', '▁'] else t, i) for s, t, i in _score_token_pairs]
    _clean_score_token_pairs = list(filter(lambda x: x[1] not in ["<s>", "</s>"], _clean_score_token_pairs))
    # _filtered_tokens = [t[1:] if t[0] == 'Ġ' else t for s, t, i in sorted(_filtered_score_token_pairs, key = lambda x: x[2])]
    # _filtered_tokens.remove("<s>") if "<s>" in _filtered_tokens else None
    # _filtered_tokens.remove("</s>") if "</s>" in _filtered_tokens else None
    # _filtered_tokens = list(filter(("<s>").__ne__, _filtered_tokens))
    # _filtered_tokens = list(filter(("</s>").__ne__, _filtered_tokens))
    # print(_clean_score_token_pairs)
    return _clean_score_token_pairs

In [14]:
def get_frugal_decompx(_score_token_pairs, percentage=80.0):
    # _tokenizer = AutoTokenizer.from_pretrained(MODEL)
    # # _tokenized_sentence = _tokenizer([_text], return_tensors="pt", padding=True, truncation=True)
    # # with torch.no_grad():
    # #     model.eval()
    # #     _, _, decompx_last_layer_outputs, _ = model(
    # #         **_tokenized_sentence,
    # #         output_attentions=False,
    # #         return_dict=False,
    # #         output_hidden_states=True,
    # #         decompx_config=CONFIGS["DecompX"]
    # #     )
    # _sentences = tokenize_sentences(_text)
    # _tokens, _attribution_scores = get_attribution_scores(_sentences, _tokenizer, model, CONFIGS["DecompX"])
    # # _tokens = _tokenizer.convert_ids_to_tokens(_tokenized_sentence["input_ids"][0])
    # # _attribution_scores = decompx_last_layer_outputs.classifier[0, :, 0].cpu().numpy() # Considering label 0
    # # _normalized_scores = normalize(_attribution_scores) #redundant
    # _score_token_pairs = sorted(list(zip(_attribution_scores, _tokens, [i for i in range(len(_tokens))])), reverse=True)
    _score_token_pairs.sort(key=lambda x: x[0], reverse=True)
    _idx = int(round((len(_score_token_pairs) * percentage) / 100.0, 0))
    _filtered_score_token_pairs = _score_token_pairs[:_idx]
    # print(_score_token_pairs)
    _filtered_tokens = [t for s, t, i in sorted(_filtered_score_token_pairs, key = lambda x: x[2])]
    # _filtered_tokens.remove("<s>") if "<s>" in _filtered_tokens else None
    # _filtered_tokens.remove("</s>") if "</s>" in _filtered_tokens else None
    # _filtered_tokens = list(filter(("<s>").__ne__, _filtered_tokens))
    # _filtered_tokens = list(filter(("</s>").__ne__, _filtered_tokens))
    return ' '.join(_filtered_tokens)

In [114]:
get_frugal_decompx(get_scores_decompx(SENTENCE), percentage=80.0)

'This is an example document for calculating attribution'

In [16]:
from global_token_attribution2.sota_models.GlobEnc.src.modeling.modeling_bert import BertForSequenceClassification
from global_token_attribution2.sota_models.GlobEnc.src.modeling.modeling_electra import ElectraForSequenceClassification
from global_token_attribution2.sota_models.GlobEnc.src.attention_rollout import AttentionRollout

# config = {"MODEL": "TehranNLP-org/bert-base-uncased-cls-sst2"}
# MODEL = "TehranNLP-org/bert-base-uncased-cls-sst2"
config = {"MODEL": "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"} # piturrolfio/bert-classifier-maths # nateraw/bert-base-uncased-ag-news google/roberta2roberta_L-24_bbc facebook/bart-large-cnn 
MODEL = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli" # yanekyuk/camembert-keyword-extractor google/roberta2roberta_L-24_bbc facebook/bart-large-cnn

In [17]:
SENTENCE = "This is an example document for calculating token attribution."
# tokenizer = AutoTokenizer.from_pretrained(config["MODEL"])
# tokenized_sentence = tokenizer.encode_plus(SENTENCE, return_tensors="pt")

# For classification tasks, the model should be a sequence classification model
# if "bert" in config["MODEL"]:
#     model = BertForSequenceClassification.from_pretrained(config["MODEL"])
# elif "electra" in config["MODEL"]:
#     model = ElectraForSequenceClassification.from_pretrained(config["MODEL"])
# else:
#     raise Exception(f"Not implemented model: {config['MODEL']}")

# For generation tasks, the model should be a Seq2Seq model
if "roberta" in config["MODEL"]:
    model = AutoModelForSequenceClassification.from_pretrained(config["MODEL"], device_map="cuda:0")
elif "electra" in config["MODEL"]:
    model = ElectraForSequenceClassification.from_pretrained(config["MODEL"], device_map="cuda:0")
elif "bert" in config["MODEL"]:
    model = BertForSequenceClassification.from_pretrained(config["MODEL"], device_map="cuda:0")
elif "bart" in config["MODEL"]:
    model = AutoModelForSeq2SeqLM.from_pretrained(config["MODEL"], device_map="cuda:0")
else:
    raise Exception(f"Not implemented model: {config['MODEL']}")
asd = model.to(device)

In [138]:
# model.eval()
# with torch.no_grad():
#     logits, attentions, norms = model(
#         **tokenized_sentence,
#         # input_ids=tokenized_sentence["input_ids"],
#         output_attentions=True,
#         output_norms=True,
#         return_dict=False
#     )
#     num_layers = len(attentions)
#     norm_nenc = (
#         torch.stack([norms[i][4] for i in range(num_layers)]).squeeze().cpu().numpy()
#     )

#     globenc = AttentionRollout().compute_flows([norm_nenc], output_hidden_states=True)[
#         0
#     ]
#     globenc = np.array(globenc)

  0%|          | 0/1 [00:00<?, ?it/s]

In [139]:
# tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"][0])

# norm_cls = globenc[:, 0, :]
# norm_cls = np.flip(norm_cls, axis=0)

# row_sums = norm_cls.max(axis=1)
# norm_cls = norm_cls / row_sums[:, np.newaxis]

# # attribution_scores = np.mean(norm_cls, axis=0)
# attribution_scores = normalize(norm_cls[0])

In [140]:
# tokens_copy = deepcopy(tokens[:])
# attribution_scores_copy = deepcopy(attribution_scores[:])
# # for i in range(len(tokens_copy)):
# i = 0
# while i < len(tokens_copy):
#     if i < len(tokens) and tokens[i][:2] == "##":
#         # print(tokens)
#         # print(i)
#         tokens[i - 1] = tokens[i - 1] + tokens[i][2:]
#         attribution_scores[i - 1] = attribution_scores[i - 1] + attribution_scores[i]
#         tokens = np.delete(tokens, i)
#         attribution_scores = np.delete(attribution_scores, i)
#         i -= 1
#         # print(tokens)
#         # print(i)
#     i += 1
# print("Token = Attribution Score")
# print("=========================")
# for token, score in zip(tokens, attribution_scores):
#     if token in ["[CLS]", "[SEP]"]:
#         continue
#     print(f"{token} = {score:.4f}")

Token = Attribution Score
[UNK] = 0.0000
is = 0.0209
an = 0.0810
example = 0.3904
document = 0.2573
for = 0.0985
calculating = 0.4693
token = 0.5440
attribution = 1.2148
. = 0.2123


In [141]:
# score_token_pairs = sorted(list(zip(attribution_scores, tokens, [i for i in range(len(tokens))])), reverse=True)
# idx = int(round((len(tokens) * 80.0) / 100.0, 0))
# filtered_score_token_pairs = score_token_pairs[:idx]
# filtered_tokens = [t for s, t, i in sorted(filtered_score_token_pairs, key = lambda x: x[2])]
# filtered_tokens.remove("[CLS]") if "[CLS]" in filtered_tokens else None
# filtered_tokens.remove("[SEP]") if "[SEP]" in filtered_tokens else None
# print(' '.join(filtered_tokens))

an example document for calculating token attribution .


In [15]:
def tokenize_sentences(text):
    """Splits the text into sentences."""
    return sent_tokenize(text)
def calculate_attribution_scores(model, tokenized_sentence, tokenizer):
    """Calculates attribution scores for a tokenized sentence."""
    with torch.no_grad():
        # model.eval()
        logits, attentions, norms = model(
            **tokenized_sentence,
            output_attentions=True,
            output_norms=True,
            return_dict=False
        )
        num_layers = len(attentions)
        norm_nenc = (
            torch.stack([norms[i][4] for i in range(num_layers)])
            .squeeze()
            .cpu()
            .numpy()
        )

        globenc = AttentionRollout().compute_flows(
            [norm_nenc], output_hidden_states=True
        )[0]
        globenc = np.array(globenc)

    norm_cls = globenc[:, 0, :]
    norm_cls = np.flip(norm_cls, axis=0)

    row_sums = norm_cls.max(axis=1)
    norm_cls = norm_cls / row_sums[:, np.newaxis]

    # attribution_scores = np.mean(norm_cls, axis=0)
    attribution_scores = normalize(norm_cls[0])

    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"][0])
    tokens_copy = deepcopy(tokens[:])
    # attribution_scores_copy = deepcopy(attribution_scores[:])

    i = 0
    while i < len(tokens_copy):
        if i < len(tokens) and tokens[i][:2] == "##":
            tokens[i - 1] = tokens[i - 1] + tokens[i][2:]
            attribution_scores[i - 1] = attribution_scores[i - 1] + attribution_scores[i]
            tokens = np.delete(tokens, i)
            attribution_scores = np.delete(attribution_scores, i)
            i -= 1
        i += 1
    normalized_scores = normalize(attribution_scores)
    # tqdm._instances.clear()
    # os.system('cls')
    # os.system('clear')
    clear_output(wait=True)

    return tokens, normalized_scores, norm_cls

def get_attribution_scores(sentences, tokenizer, model):
    """Calculates attribution scores for a list of sentences."""
    all_tokens = []
    all_attribution_scores = []
    all_norm_cls = []

    for sentence in sentences:
        tokenized_sentence = tokenizer.encode_plus(sentence, return_tensors="pt", verbose=False).to(device)
        tokens, attribution_scores, norm_cls = calculate_attribution_scores(
            model, tokenized_sentence, tokenizer
        )

        all_tokens.append(tokens)
        all_attribution_scores.append(attribution_scores)
        all_norm_cls.append(norm_cls)

    return all_tokens, all_attribution_scores, all_norm_cls

def get_attribution_scores_with_sentence_importance(sentences, tokenizer, model):
    """Calculates attribution scores for a list of sentences."""
    all_tokens = []
    all_attribution_scores = []
    all_norm_cls = []

    # Calculate sentence importance scores for sentences using LexRank
    lexrank = LexRank(" ".join(sentences))
    sentence_scores = lexrank.rank_sentences(sentences)

    for i, sentence in enumerate(sentences):
        tokenized_sentence = tokenizer.encode_plus(
            sentence, padding='longest', truncation=True, return_tensors="pt", verbose=False
        ).to(device)
        tokens, attribution_scores, norm_cls = calculate_attribution_scores(
            model, tokenized_sentence, tokenizer
        )

        attribution_scores *= sentence_scores[i]
        norm_cls *= sentence_scores[i]

        all_tokens.extend(tokens)
        all_attribution_scores.extend(attribution_scores)
        all_norm_cls.extend(norm_cls)

    return all_tokens, all_attribution_scores, all_norm_cls



In [16]:
def get_scores_globenc(_text):
    _tokenizer = AutoTokenizer.from_pretrained(MODEL)
    # _tokenized_sentence = _tokenizer([_text], return_tensors="pt", padding=True)
    _sentences = tokenize_sentences(_text)
    _tokens, _attribution_scores, _ = get_attribution_scores_with_sentence_importance(_sentences, _tokenizer, model)
    _score_token_pairs = sorted(list(zip(_attribution_scores, _tokens, [i for i in range(len(_tokens))])), reverse=True)
    _clean_score_token_pairs = list(filter(lambda x: x[1] not in ["[CLS]", "[SEP]"], _score_token_pairs))
    return _clean_score_token_pairs

In [17]:
def get_frugal_globenc(_score_token_pairs, percentage=80.0):
    # _tokenizer = AutoTokenizer.from_pretrained(MODEL)
    # _tokenized_sentence = _tokenizer([_text], return_tensors="pt", padding=True)
    # with torch.no_grad():
    #     logits, attentions, norms = model(
    #         **tokenized_sentence,
    #         output_attentions=True,
    #         output_norms=True,
    #         return_dict=False
    #     )
    #     _num_layers = len(attentions)
    #     _norm_nenc = (
    #         torch.stack([norms[i][4] for i in range(_num_layers)]).squeeze().cpu().numpy()
    #     )

    #     _globenc = AttentionRollout().compute_flows([_norm_nenc], output_hidden_states=True)[0]
    #     _globenc = np.array(_globenc)
    # _tokens = _tokenizer.convert_ids_to_tokens(_tokenized_sentence["input_ids"][0])
    # _norm_cls = _globenc[:, 0, :]
    # _norm_cls = np.flip(_norm_cls, axis=0)
    # _row_sums = _norm_cls.max(axis=1)
    # _norm_cls = _norm_cls / _row_sums[:, np.newaxis]
    # # attribution_scores = np.mean(norm_cls, axis=0)
    # _attribution_scores = normalize(_norm_cls[0])
    # _tokens_copy = deepcopy(_tokens[:])
    # _attribution_scores_copy = deepcopy(_attribution_scores[:])
    # # for i in range(len(_tokens_copy)):
    # i = 0
    # while i < len(_tokens_copy):
    #     if i < len(_tokens) and _tokens[i][:2] == "##":
    #         _tokens[i - 1] = _tokens[i - 1] + _tokens[i][2:]
    #         _attribution_scores[i - 1] = _attribution_scores[i - 1] + _attribution_scores[i]
    #         _tokens = np.delete(_tokens, i)
    #         _attribution_scores = np.delete(_attribution_scores, i)
    #         i -= 1
    #     i += 1
    # _normalized_scores = normalize(_attribution_scores)
    # _score_token_pairs = sorted(list(zip(_normalized_scores, _tokens, [i for i in range(len(_tokens))])), reverse=True)
    _score_token_pairs.sort(key=lambda x: x[0], reverse=True)
    _idx = int(round((len(_score_token_pairs) * percentage) / 100.0, 0))
    _filtered_score_token_pairs = _score_token_pairs[:_idx]
    _filtered_tokens = [t for s, t, i in sorted(_filtered_score_token_pairs, key = lambda x: x[2])]
    # _filtered_tokens.remove("[CLS]") if "[CLS]" in _filtered_tokens else None
    # _filtered_tokens.remove("[SEP]") if "[SEP]" in _filtered_tokens else None
    return ' '.join(_filtered_tokens)

In [156]:
get_frugal_globenc(get_scores_globenc(SENTENCE.lower()), percentage=80.0)

'an example document for calculating token attribution .'

In [18]:
def select_random_indices(dataset, num_to_keep, random_seed=42):
    indices = list(range(dataset.num_rows))
    random.seed(random_seed)
    random.shuffle(indices)
    return indices[:num_to_keep]

In [98]:
dataset_gsm = load_dataset("openai/gsm8k", "main")

reduction_rate    = 0.042 # 4% of the original dataset = 1000 examples
# reduction_rate    = 0.0001 # 0.01% of the original dataset = 2 examples (for testing)
num_train_to_keep = int(reduction_rate * dataset_gsm["train"].num_rows)
num_test_to_keep  = 1000 # int(reduction_rate * dataset_news["test"].num_rows)

train_indices = select_random_indices(dataset_gsm["train"], num_train_to_keep)
test_indices  = select_random_indices(dataset_gsm["test"], num_test_to_keep)

dataset_gsm  = DatasetDict({
    "train": dataset_gsm["train"].select(train_indices),
    "test": dataset_gsm["test"].select(test_indices),
})

dataset_gsm

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 313
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1000
    })
})

In [101]:
df_train_gsm = pd.DataFrame(dataset_gsm['train'])
df_test_gsm = pd.DataFrame(dataset_gsm['test'])

In [13]:
import replicate

def md(t):
  display(Markdown(t))

def llama2(prompt, temperature=0.0, input_print=True):
  output = replicate.run(
    "meta/llama-2-7b-chat",
    input={
        "prompt": prompt,
        "max_tokens": 2048,
        "temperature": temperature})
  return "".join(output)

def llama3_8b(prompt, temperature=0.0):
  output = replicate.run(
    "meta/meta-llama-3-8b-instruct",
    input={
        "prompt": prompt,
        "max_tokens": 2048,
        "temperature": temperature})
  return "".join(output)

def llama3_70b(prompt, temperature=0.0):
  output = replicate.run(
    "meta/meta-llama-3-70b-instruct",
    input={
        "prompt": prompt,
        "max_tokens": 2048,
        "temperature": temperature})
  return "".join(output)

In [14]:
prompt_math_reasoning = """As an expert problem solver solve step by step the following mathematical question.
After performing the correct set of steps for solving the problem, please fill in the <OUTPUT> tag with your numerical answer.\n\nQuestion:\n\n"""

In [21]:
def get_prompt(train_data, model_name, shots=4, random_seed=42, percentage=80.0):
    assert shots % 2 == 0
    prompt = "Your task is to perform sentiment classification of movie reviews. Below are some examples of movie reviews and their corresponding sentiment labels (Positive/Negative):\n\n" # Needs changing for each dataset.
    classes = train_data['label'].nunique()
    # print(classes)
    classwise_examples = []
    for i in range(classes):
        examples = [example for index, example in train_data.iterrows() if example['label'] == i]
        classwise_examples.append(examples)
    samples = []
    for _class in classwise_examples:
        sample = random.sample(_class, shots // classes)
        samples += sample
    random.seed(random_seed)
    random.shuffle(samples)

    # Needs changing for each dataset.
    for example in samples:
        review = example['text']
        if model_name == "DecompX":
            frugal_review = get_frugal_decompx(get_scores_decompx(review), percentage)
        elif model_name == "GlobEnc":
            frugal_review = get_frugal_globenc(get_scores_globenc(review), percentage)
        else:
            frugal_review = get_frugal(review, percentage)
        label = "Positive" if example['label'] == 1 else "Negative"
        prompt += f"Review: {frugal_review}\nSentiment: {label}\n\n"
    prompt += "Now, with a single label, classify the sentiment of the following review by filling in the <OUTPUT> tag:\n"
    return prompt

In [22]:
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}
target_names = ['Negative', 'Positive']

In [102]:
clean_score_token_idx_tuples = []
start = time.time()
prevIndex = 0
with alive_bar(len(df_test_gsm), force_tty=True) as bar:
    for index, example in df_test_gsm.iterrows():
        clean_score_token_idx_tuples.append(get_scores_decompx(example['question']))
        if index != 0 and index % 50 == 0:
            print(f"Processed {prevIndex} – {index} indexed examples in {format_timespan(time.time() - start)}.")
            start = time.time()
            prevIndex = index + 1
        bar()
df_test_scores = df_test_gsm.copy()
df_test_scores['score_token_idx_tuples'] = clean_score_token_idx_tuples

on 50: Processed 0 – 50 indexed examples in 1 minute and 12.26 seconds.         
on 100: Processed 51 – 100 indexed examples in 1 minute and 8.57 seconds.       
on 150: Processed 101 – 150 indexed examples in 1 minute and 6.7 seconds.       
on 200: Processed 151 – 200 indexed examples in 1 minute and 6.27 seconds.      
on 250: Processed 201 – 250 indexed examples in 1 minute and 10.89 seconds.     
on 300: Processed 251 – 300 indexed examples in 1 minute and 10.97 seconds.     
on 350: Processed 301 – 350 indexed examples in 1 minute and 15.53 seconds.     
on 400: Processed 351 – 400 indexed examples in 1 minute and 14.84 seconds.     
on 450: Processed 401 – 450 indexed examples in 1 minute and 10.12 seconds.     
on 500: Processed 451 – 500 indexed examples in 1 minute and 12.03 seconds.     
on 550: Processed 501 – 550 indexed examples in 1 minute and 7.88 seconds.      
on 600: Processed 551 – 600 indexed examples in 1 minute and 8.97 seconds.      
on 650: Processed 601 – 650 

In [109]:
clean_score_token_idx_tuples = []
start = time.time()
prevIndex = 0
with alive_bar(len(df_test_gsm), force_tty=True) as bar:
    for index, example in df_test_gsm.iterrows():
        clean_score_token_idx_tuples.append(get_scores_globenc(example['question']))
        if index != 0 and index % 50 == 0:
            print(f"Processed {prevIndex} – {index} indexed examples in {format_timespan(time.time() - start)}.")
            start = time.time()
            prevIndex = index + 1
        bar()
df_test_scores = df_test_gsm.copy()
df_test_scores['score_token_idx_tuples'] = clean_score_token_idx_tuples

|████████████████████████████████████████| 1000/1000 [100%] in 3:52:26.2 (0.07/s


In [110]:
df_test_scores.to_json('scores/globenc/gsm8k_test.json', orient='records')

In [103]:
df_test_scores.to_json('scores/decompx/gsm8k_test.json', orient='records')

In [15]:
def find_numbers(x: str) -> list[str]:
  """Finds all numbers in a string."""
  # Search for number, possibly negative (hyphen), with thousand separators
  # (comma), and with a decimal point (period inbetween digits).
  numbers = re.compile(
      r'-?[\d,]*\.?\d+',
      # r'-?[\d,]*\.?\d+(?:[a-zA-Z]+)?',
      re.MULTILINE | re.DOTALL | re.IGNORECASE,
  ).findall(x)
  return numbers

In [226]:
frugal_examples = []
with open('scores/decompx/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['context'])
        # for i in range(len(sample['score_token_idx_tuples'])):
            # if sample['score_token_idx_tuples'][i][1] in numbers:
            #     sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_decompx(sample['score_token_idx_tuples'], percentage=80.0))
df_test_80 = df_test_cosmos.copy()
df_test_80['frugal_text'] = frugal_examples
df_test_80.to_json('scores/decompx/cosmosqa_test_80.json', orient='records')

In [227]:
frugal_examples = []
with open('scores/decompx/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['question'])
        # for i in range(len(sample['score_token_idx_tuples'])):
        #     if sample['score_token_idx_tuples'][i][1] in numbers:
        #         sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_decompx(sample['score_token_idx_tuples'], percentage=60.0))
df_test_60 = df_test_cosmos.copy()
df_test_60['frugal_text'] = frugal_examples
df_test_60.to_json('scores/decompx/cosmosqa_test_60.json', orient='records')

In [228]:
frugal_examples = []
with open('scores/decompx/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['question'])
        # for i in range(len(sample['score_token_idx_tuples'])):
        #     if sample['score_token_idx_tuples'][i][1] in numbers:
        #         sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_decompx(sample['score_token_idx_tuples'], percentage=50.0))
df_test_50 = df_test_cosmos.copy()
df_test_50['frugal_text'] = frugal_examples
df_test_50.to_json('scores/decompx/cosmosqa_test_50.json', orient='records')

In [229]:
frugal_examples = []
with open('scores/globenc/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['question'])
        # for i in range(len(sample['score_token_idx_tuples'])):
        #     if sample['score_token_idx_tuples'][i][1] in numbers:
        #         sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_globenc(sample['score_token_idx_tuples'], percentage=80.0))
df_test_80 = df_test_cosmos.copy()
df_test_80['frugal_text'] = frugal_examples
df_test_80.to_json('scores/globenc/cosmosqa_test_80.json', orient='records')

In [230]:
frugal_examples = []
with open('scores/globenc/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['question'])
        # for i in range(len(sample['score_token_idx_tuples'])):
        #     if sample['score_token_idx_tuples'][i][1] in numbers:
        #         sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_globenc(sample['score_token_idx_tuples'], percentage=60.0))
df_test_60 = df_test_cosmos.copy()
df_test_60['frugal_text'] = frugal_examples
df_test_60.to_json('scores/globenc/cosmosqa_test_60.json', orient='records')

In [231]:
frugal_examples = []
with open('scores/globenc/cosmosqa_test.json') as data_file:
    d = json.load(data_file)
    for sample in d:
        # numbers = find_numbers(sample['question'])
        # for i in range(len(sample['score_token_idx_tuples'])):
        #     if sample['score_token_idx_tuples'][i][1] in numbers:
        #         sample['score_token_idx_tuples'][i][0] = np.inf
        frugal_examples.append(get_frugal_globenc(sample['score_token_idx_tuples'], percentage=50.0))
df_test_50 = df_test_cosmos.copy()
df_test_50['frugal_text'] = frugal_examples
df_test_50.to_json('scores/globenc/cosmos_test_50.json', orient='records')

In [9]:
# from openai import OpenAI
# client = OpenAI(
#     # Defaults to os.environ.get("OPENAI_API_KEY")
#     api_key=os.environ.get("OPENAI_API_KEY"),
# )
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [16]:
@backoff.on_exception(backoff.expo, (openai.OpenAIError, openai.APIError), max_tries=5)
def get_openai_response(prompt, model="o3-mini"):
    messages = [{"role": "user", "content": prompt + '\n' + '<OUTPUT>'}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.0, # this is the degree of randomness of the model's output
        reasoning_effort="medium"
    )
    return response.choices[0].message.content

In [17]:
def get_gemini_response(prompt, model="gemini-2.0-flash-thinking-exp-01-21"): # gemini-2.0-flash
    client = genai.Client(api_key=GEMINI_API_KEY)
    contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_text(text=prompt),
                ],
            ),
        ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.7,
        top_p=0.95,
        top_k=64,
        max_output_tokens=65536,
        response_mime_type="text/plain",
    )
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=generate_content_config,
    )
    return response.text
    # print(response.text)

In [42]:
# @backoff.on_exception(backoff.expo, (openai.OpenAIError, openai.APIError), max_tries=5)
def get_kimi_response(prompt, model="kimi-k1.5-preview"):
    # messages = [{"role": "user", "content": prompt + '\n' + '<OUTPUT>'}]
    client = Client(
        api_key=KIMI_API_KEY,
        base_url="https://api.moonshot.ai/v1",
    )
    messages = [
        {
            "role": "user",
            "content": prompt,
        },
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.3, # this is the degree of randomness of the model's output
        stream=True,
        max_tokens=8192,
    )
    res = ""
    for chunk in response:
        if chunk.choices[0].delta:
            if chunk.choices[0].delta.content:
                res += chunk.choices[0].delta.content
                # print(chunk.choices[0].delta.content, end="")
    # return response.choices[0].message.content
    return res

In [50]:
# get_kimi_response("Hello, how are you today?")
# get_kimi_response("The lengths of the two legs of a right triangle are 3 cm and 4 cm respectively. Find the length of the hypotenuse of this right triangle.")
get_kimi_response(prompt_summarization + "WASHINGTON (Reuters) - U.S. President Donald Trump said on Thursday a major conflict with North Korea is possible in the standoff over its nuclear and missile programs, but he would prefer a diplomatic outcome to the dispute.  \u201cThere is a chance that we could end up having a major, major conflict with North Korea. Absolutely,\u201d Trump told Reuters in an Oval Office interview ahead of his 100th day in office on Saturday.  Nonetheless, Trump said he wanted to peacefully resolve a crisis that has bedeviled multiple U.S. presidents, a path that he and his administration are emphasizing by preparing a variety of new economic sanctions while not taking the military option off the table.  \u201cWe\u2019d love to solve things diplomatically but it\u2019s very difficult,\u201d he said.  REUTERS RECOMMENDS Racism on the rise: Reuters poll How North Korea gets its oil from China In other highlights of the 42-minute interview, Trump was cool to speaking again with Taiwan\u2019s president after an earlier telephone call with her angered China.  He also said he wants South Korea to pay the cost of the U.S. THAAD anti-missile defense system, which he estimated at $1 billion, and intends to renegotiate or terminate a U.S. free trade pact with South Korea because of a deep trade deficit with Seoul.  Asked when he would announce his intention to renegotiate the pact, Trump said: \u201cVery soon. I\u2019m announcing it now.\u201d  Trump also said he was considering adding stops to Israel and Saudi Arabia to a Europe trip next month, emphasizing that he wanted to see an Israeli-Palestinian peace. He complained that Saudi Arabia was not paying its fair share for U.S. defense.  Asked about the fight against Islamic State, Trump said the militant group had to be defeated.  \u201cI have to say, there is an end. And it has to be humiliation,\u201d he said, when asked about what the endgame was for defeating Islamist violent extremism.  Trump said North Korea was his biggest global challenge. He lavished praise on Chinese President Xi Jinping for Chinese assistance in trying to rein in Pyongyang. The two leaders met in Florida earlier this month.  \u201cI believe he is trying very hard. He certainly doesn\u2019t want to see turmoil and death. He doesn\u2019t want to see it. He is a good man. He is a very good man and I got to know him very well.  \u201cWith that being said, he loves China and he loves the people of China. I know he would like to be able to do something, perhaps it\u2019s possible that he can\u2019t,\u201d Trump said.  Trump spoke just a day after he and his top national security advisers briefed U.S. lawmakers on the North Korean threat and one day before Secretary of State Rex Tillerson will press the United Nations Security Council on sanctions to further isolate Pyongyang over its nuclear and missile programs.  The Trump administration on Wednesday declared North Korea \u201can urgent national security threat and top foreign policy priority.\u201d It said it was focusing on economic and diplomatic pressure, including Chinese cooperation in containing its defiant neighbor and ally, and remained open to negotiations.  U.S. officials said military strikes remained an option but played down the prospect, though the administration has sent an aircraft carrier and a nuclear-powered submarine to the region in a show of force.  Any direct U.S. military action would run the risk of massive North Korean retaliation and huge casualties in Japan and South Korea and among U.S. forces in both countries.  Trump, asked if he considered North Korean leader Kim Jong Un to be rational, said he was operating from the assumption that he is rational. He noted that Kim had taken over his country at an early age.  \u201cHe\u2019s 27 years old. His father dies, took over a regime. So say what you want but that is not easy, especially at that age.  \u201cI\u2019m not giving him credit or not giving him credit, I\u2019m just saying that\u2019s a very hard thing to do. As to whether or not he\u2019s rational, I have no opinion on it. I hope he\u2019s rational,\u201d he said.  Trump, sipping a Coke delivered by an aide after the president ordered it by pressing a button on his desk, rebuffed an overture from Taiwan President Tsai Ing-wen, who told Reuters a direct phone call with Trump could take place again after their first conversation in early December angered Beijing.  China considers neighboring Taiwan to be a renegade province.  \u201cMy problem is that I have established a very good personal relationship with President Xi,\u201d said Trump. \u201cI really feel that he is doing everything in his power to help us with a big situation. So I wouldn\u2019t want to be causing difficulty right now for him.  \u201cSo I would certainly want to speak to him first.\u201d  Trump also said he hoped to avoid a potential government shutdown amid a dispute between congressional Republicans and Democrats over a spending deal with a Saturday deadline looming.  But he said if a shutdown takes place, it will be the Democrats\u2019 fault for trying to add money to the legislation to \u201cbail out Puerto Rico\u201d and other items.  He also defended the one-page tax plan he unveiled on Wednesday from criticism that it would increase the U.S. deficit, saying better trade deals and economic growth would offset the costs.  \u201cWe will do trade deals that are going to make up for a tremendous amount of the deficit. We are going to be doing trade deals that are going to be much better trade deals,\u201d Trump said.")

BadRequestError: Error code: 400 - {'error': {'code': 400, 'message': 'The request was rejected because it was considered high risk', 'param': 'prompt', 'type': 'content_filter', 'innererror': {}}}

In [18]:
def predict(input_text, att_model_name, shots=4, random_seed=42, percentage=80.0):
    # prompt = get_prompt(df_train, att_model_name, shots, random_seed, percentage)
    # prompt = prompt_4_shot_decompx + input_text + '\n' + '<OUTPUT>'
    # prompt = prompt_4_shot_globenc + input_text + '\n' + '<OUTPUT>'
    prompt = prompt_math_reasoning + input_text + '\n' + 'Answer: <OUTPUT>'
    # prompt = prompt_cosmosqa + input_text # + '\n' + 'Answer: <OUTPUT>'
    output = get_gemini_response(prompt) # get_openai_response(prompt) # llama3_70b(prompt) # llama3_8b(prompt)   # Change model here.
    # prediction = None
    # if output.find('Positive') != -1:
    #     prediction = 1
    # else:
    #     prediction = 0
    return prompt, output#, prediction

In [ ]:
with open('scores/decompx/gsm8k_test_80.json') as data_file:
    d = json.load(data_file)
df_test_80 = pd.DataFrame.from_dict(d)
test_data = df_test_80.copy() #change this to test_80, test_60, test_50
prompts = []
responses = []
preds = []
start = time.time()
prevIndex = 0
with alive_bar(len(test_data), force_tty=True) as bar:
    for index, row in test_data.iterrows():
        while True:
            try:
                prompt, response = predict(row['frugal_text'], att_model_name="DecompX", shots=4, random_seed=42, percentage=80.0)
                # prompt, response = predict('Context:\n'+row['frugal_text']+'\nQuestion: '+row['question']+'\nChoices:\n'+'1) '+
                #                    row['answer0']+'\n2) '+row['answer1']+'\n3) '+row['answer2']+'\n4) '+row['answer3'], 
                #                    att_model_name="DecompX", shots=4, random_seed=42, percentage=60.0)
                break
            except:
                continue
        prompts.append(prompt)
        responses.append(response)
        # preds.append(pred)
        if index != 0 and index % 50 == 0:
            print(f"Processed {prevIndex} – {index} indexed examples in {format_timespan(time.time() - start)}.")
            start = time.time()
            prevIndex = index + 1
        bar()
        # if index == 5:
        #     break
test_data['prompt'] = prompts
test_data['response'] = responses
# test_data['pred'] = preds

|                                        | ▁▃▅ 0/1000 [0%] in 0s (~0s, 0.0/s) 

on 50: Processed 0 – 50 indexed examples in 6 minutes and 36.69 seconds.        
on 100: Processed 51 – 100 indexed examples in 5 minutes and 13.86 seconds.     
on 150: Processed 101 – 150 indexed examples in 5 minutes and 24.95 seconds.    
on 200: Processed 151 – 200 indexed examples in 5 minutes and 39.07 seconds.    
on 250: Processed 201 – 250 indexed examples in 7 minutes and 36.66 seconds.    
on 300: Processed 251 – 300 indexed examples in 6 minutes and 48.61 seconds.    
on 350: Processed 301 – 350 indexed examples in 5 minutes and 35.57 seconds.    
on 400: Processed 351 – 400 indexed examples in 6 minutes and 3.89 seconds.     
on 450: Processed 401 – 450 indexed examples in 6 minutes and 10.45 seconds.    
on 500: Processed 451 – 500 indexed examples in 5 minutes and 57.55 seconds.    
on 550: Processed 501 – 550 indexed examples in 8 hours, 25 minutes and 38.21 seconds.
on 600: Processed 551 – 600 indexed examples in 7 minutes and 6.08 seconds.     
on 650: Processed 601 

In [ ]:
test_data.to_json('responses/decompx/gemini2.0_flash_thinking_gsm8k_test_80.json', orient='records')

In [ ]:
with open('scores/decompx/gsm8k_test_60.json') as data_file:
    d = json.load(data_file)
df_test_60 = pd.DataFrame.from_dict(d)
test_data = df_test_60.copy() #change this to test_80, test_60, test_50
prompts = []
responses = []
preds = []
start = time.time()
prevIndex = 0
with alive_bar(len(test_data), force_tty=True) as bar:
    for index, row in test_data.iterrows():
        while True:
            try:
                prompt, response = predict(row['frugal_text'], att_model_name="DecompX", shots=4, random_seed=42, percentage=60.0)
                # prompt, response = predict('Context:\n'+row['frugal_text']+'\nQuestion: '+row['question']+'\nChoices:\n'+'1) '+
                #                    row['answer0']+'\n2) '+row['answer1']+'\n3) '+row['answer2']+'\n4) '+row['answer3'], 
                #                    att_model_name="DecompX", shots=4, random_seed=42, percentage=60.0)
                break
            except:
                continue
        prompts.append(prompt)
        responses.append(response)
        # preds.append(pred)
        if index != 0 and index % 50 == 0:
            print(f"Processed {prevIndex} – {index} indexed examples in {format_timespan(time.time() - start)}.")
            start = time.time()
            prevIndex = index + 1
        bar()
        # if index == 5:
        #     break
test_data['prompt'] = prompts
test_data['response'] = responses
# test_data['pred'] = preds

on 50: Processed 0 – 50 indexed examples in 8 minutes and 37.84 seconds.        
on 100: Processed 51 – 100 indexed examples in 7 minutes and 58.04 seconds.     
on 150: Processed 101 – 150 indexed examples in 7 minutes and 58.95 seconds.    
on 200: Processed 151 – 200 indexed examples in 8 minutes and 23.8 seconds.     
on 250: Processed 201 – 250 indexed examples in 8 minutes and 38.01 seconds.    
on 300: Processed 251 – 300 indexed examples in 7 minutes and 22.01 seconds.    
on 350: Processed 301 – 350 indexed examples in 11 minutes and 8.79 seconds.    
on 400: Processed 351 – 400 indexed examples in 7 minutes and 22.37 seconds.    
on 450: Processed 401 – 450 indexed examples in 10 minutes and 14.39 seconds.   
on 500: Processed 451 – 500 indexed examples in 8 minutes and 1.06 seconds.     
on 550: Processed 501 – 550 indexed examples in 6 minutes and 59.78 seconds.    
on 600: Processed 551 – 600 indexed examples in 8 minutes and 9.3 seconds.      
on 650: Processed 601 – 650 

In [ ]:
test_data.to_json('responses/decompx/gemini2.0_flash_thinking_gsm8k_test_60.json', orient='records')

In [ ]:
with open('scores/decompx/gsm8k_test_50.json') as data_file:
    d = json.load(data_file)
df_test_50 = pd.DataFrame.from_dict(d)
test_data = df_test_50.copy() #change this to test_80, test_60, test_50
prompts = []
responses = []
preds = []
start = time.time()
prevIndex = 0
with alive_bar(len(test_data), force_tty=True) as bar:
    for index, row in test_data.iterrows():
        while True:
            try:
                prompt, response = predict(row['frugal_text'], att_model_name="DecompX", shots=4, random_seed=42, percentage=50.0)
                # prompt, response = predict('Context:\n'+row['frugal_text']+'\nQuestion: '+row['question']+'\nChoices:\n'+'1) '+
                #                    row['answer0']+'\n2) '+row['answer1']+'\n3) '+row['answer2']+'\n4) '+row['answer3'], 
                #                    att_model_name="DecompX", shots=4, random_seed=42, percentage=50.0)
                break
            except:
                continue
        prompts.append(prompt)
        responses.append(response)
        # preds.append(pred)
        if index != 0 and index % 50 == 0:
            print(f"Processed {prevIndex} – {index} indexed examples in {format_timespan(time.time() - start)}.")
            start = time.time()
            prevIndex = index + 1
        bar()
        # if index == 5:
        #     break
test_data['prompt'] = prompts
test_data['response'] = responses
# test_data['pred'] = preds

on 50: Processed 0 – 50 indexed examples in 8 minutes and 54.75 seconds.        
on 100: Processed 51 – 100 indexed examples in 8 minutes and 26.72 seconds.     
on 150: Processed 101 – 150 indexed examples in 10 minutes and 19.61 seconds.   
on 200: Processed 151 – 200 indexed examples in 8 minutes and 25.47 seconds.    
on 250: Processed 201 – 250 indexed examples in 10 minutes and 5.85 seconds.    
on 300: Processed 251 – 300 indexed examples in 8 minutes and 4.91 seconds.     
on 350: Processed 301 – 350 indexed examples in 9 minutes and 35.75 seconds.    
on 400: Processed 351 – 400 indexed examples in 8 minutes and 0.24 seconds.     
on 450: Processed 401 – 450 indexed examples in 8 minutes and 35.69 seconds.    
on 500: Processed 451 – 500 indexed examples in 8 minutes and 59.03 seconds.    
on 550: Processed 501 – 550 indexed examples in 11 minutes and 24.62 seconds.   
on 600: Processed 551 – 600 indexed examples in 7 minutes and 50.32 seconds.    
on 650: Processed 601 – 650 

In [ ]:
test_data.to_json('responses/decompx/gemini2.0_flash_thinking_gsm8k_test_50.json', orient='records')

In [52]:
from huggingface_hub import login
login(config['HUGGING_FACE_TOKEN']) # put your User Access Tokens here

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/cse/.cache/huggingface/token
Login successful
